In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import recall_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
import string

## 1. Load the dataset

In [ ]:
# load datasets given
df_movie_details = pd.read_json("../data/IMDB_movie_details.json", lines = True)
df_reviews = pd.read_json("../data/IMDB_reviews.json", lines = True)

In [ ]:
df_movie_details.info()

In [ ]:
df_reviews.info()

In [ ]:
df_reviews['is_spoiler'].value_counts()

In [ ]:
df_reviews = pd.read_pickle("../data/tokenized_reviews.pkl.gz", compression = 'gzip')

## 2. Text Preprocessing

### Tokenize review texts

In [ ]:
# TOKENIZING WILL TAKE ~15 MINUTES
# If you want to save after tokenizing feel free to do so to save time in tokenizing again
df_reviews['tokenized_summary'] = list(map(word_tokenize, df_reviews['review_summary']))
df_reviews['tokenized_reviews'] = list(map(word_tokenize, df_reviews['review_text']))

### Removing stop words and punctuations from the list

In [ ]:
# get stop words
stop_words_and_punctuations = set(stopwords.words('english') + list(string.punctuation))

In [ ]:
# remove stop words and punctuations from the tokenized list
df_reviews['tokenized_summary'] = list(map(lambda x: [word.lower() for word in x if word.lower() not in stop_words_and_punctuations], df_reviews['tokenized_summary']))
df_reviews['tokenized_reviews'] = list(map(lambda x: [word.lower() for word in x if word.lower() not in stop_words_and_punctuations], df_reviews['tokenized_reviews']))

### Stemming or Lemmatisation

In [ ]:
# stem or lemmatise words
stemmer = PorterStemmer()
df_reviews['tokenized_summary'] = list(map(lambda x: [stemmer.stem(word) for word in x], df_reviews['tokenized_summary']))
df_reviews['tokenized_reviews'] = list(map(lambda x: [stemmer.stem(word) for word in x], df_reviews['tokenized_reviews']))

### Return back the cleaned tokenized words into text

In [ ]:
df_reviews["text_tokenized"] = list(map(lambda x: ' '.join(x), df_reviews['tokenized_reviews']))

### Save the new dataset

In [ ]:
# save changes made to original dataset to save time tokenizing etc
df_reviews.to_pickle("../data/cleaned_reviews.pkl.gz", compression = 'gzip')

### Undersample non-spoiler movies

In [ ]:
df_movie_details["release_date"] = pd.to_datetime(df_movie_details["release_date"])

In [ ]:
(df_movie_details["release_date"] < "2015-01-01").value_counts()

In [ ]:
train_movies = df_movie_details["movie_id"].loc[df_movie_details["release_date"] < "2015-01-01"]
test_movies = df_movie_details["movie_id"].loc[df_movie_details["release_date"] >= "2015-01-01"]

In [ ]:
df_reviews.groupby('movie_id')["is_spoiler"].value_counts()["tt0012349"]

In [ ]:
df_train = df_reviews.loc[df_reviews["movie_id"].isin(train_movies)]
df_test = df_reviews.loc[df_reviews["movie_id"].isin(test_movies)]

In [ ]:
df_train_spoiler = df_train.loc[df_reviews['is_spoiler'] == True]
df_train_non_spoiler = df_train.loc[df_reviews['is_spoiler'] == False]

df_train_non_spoiler = df_train_non_spoiler.sample(n = df_train_spoiler["is_spoiler"].count(), random_state = 42)
df_train = pd.concat([df_train_spoiler, df_train_non_spoiler])

df_test_spoiler = df_test.loc[df_reviews['is_spoiler'] == True]
df_test_non_spoiler = df_test.loc[df_reviews['is_spoiler'] == False]

df_test_non_spoiler = df_test_non_spoiler.sample(n = df_test_spoiler["is_spoiler"].count(), random_state = 42)
df_test = pd.concat([df_test_spoiler, df_test_non_spoiler])

In [ ]:
df_test['is_spoiler'].value_counts()

In [ ]:
df_train['is_spoiler'].value_counts()

## 3. Model Building

### Bag of words

In [ ]:
vectorizer = CountVectorizer()
bow_train = vectorizer.fit_transform(df_train["text_tokenized"])
bow_test = vectorizer.transform(df_test["review_text"])

In [ ]:
logistic_model = LogisticRegression(max_iter = 1e5)
logistic_model.fit(bow_train, df_train["is_spoiler"])

In [ ]:
logistic_model.score(bow_test, df_test["is_spoiler"])

In [ ]:
print(confusion_matrix(df_test["is_spoiler"], logistic_model.predict(bow_test)))

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(df_test["is_spoiler"], logistic_model.predict(bow_test))).plot()

In [ ]:
recall_score(df_test["is_spoiler"], logistic_model.predict(bow_test))
print(classification_report(df_test["is_spoiler"], logistic_model.predict(bow_test)))

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel = 'sigmoid', max_iter = 1e4)
clf.fit(bow_train, df_train["is_spoiler"])

In [ ]:
clf.score(bow_test, df_test["is_spoiler"])

In [ ]:
bow_train

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.InputLayer(input_shape = (bow_train.shape[1],), sparse = True),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
coo = bow_train.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
sparse_train = tf.SparseTensor(indices, coo.data, coo.shape)

coo = bow_test.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
sparse_test = tf.SparseTensor(indices, coo.data, coo.shape)

In [ ]:
model.fit(sparse_train, df_train["is_spoiler"].to_numpy(), validation_data = (sparse_test, df_test["is_spoiler"].to_numpy()), epochs = 2)

In [ ]:
loss, accuracy = model.evaluate(sparse_train, df_train["is_spoiler"].to_numpy(), verbose = False)
accuracy

In [ ]:
model.evaluate(sparse_test, df_test["is_spoiler"].to_numpy(), verbose = False)

In [ ]:
y_pred = model.predict(sparse_test, verbose = False) > 0.5
y_pred = y_pred.astype('int')

In [ ]:
print(classification_report(df_test["is_spoiler"], y_pred))

In [ ]:
print(confusion_matrix(df_test["is_spoiler"], y_pred))

### Tf-idf

In [ ]:
tfidfvectorizer = TfidfVectorizer()
tfidf_train = tfidfvectorizer.fit_transform(df_train['text_tokenized'])
tfidf_test = tfidfvectorizer.transform(df_test['text_tokenized'])

In [ ]:
logistic_model = LogisticRegression(max_iter = 1e5)
logistic_model.fit(tfidf_train, df_train["is_spoiler"])

In [ ]:
logistic_model.score(tfidf_test, df_test["is_spoiler"])

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='sigmoid', max_iter = 1e4)
clf.fit(tfidf_train, df_train['is_spoiler'])

In [ ]:
clf.score(tfidf_test, df_test['is_spoiler'])

In [ ]:
tfidf_train

In [ ]:
tfidf_coo_train = tfidf_train.tocoo()
indices = np.mat([tfidf_coo_train.row, tfidf_coo_train.col]).transpose()
tfidf_sparse_train = tf.SparseTensor(indices, tfidf_coo_train.data, tfidf_coo_train.shape)

tfidf_coo_test = tfidf_test.tocoo()
indices = np.mat([tfidf_coo_test.row, tfidf_coo_test.col]).transpose()
tfidf_sparse_test = tf.SparseTensor(indices, tfidf_coo_test.data, tfidf_coo_test.shape)

In [ ]:
tfidf_model = tf.keras.models.Sequential([tf.keras.layers.InputLayer(input_shape = (tfidf_train.shape[1],), sparse = True),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
tfidf_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
tfidf_model.fit(tfidf_sparse_train, df_train["is_spoiler"].to_numpy(), validation_data = (tfidf_sparse_test, df_test["is_spoiler"].to_numpy()), epochs = 2)

In [ ]:
print(classification_report(df_test['is_spoiler'], model.predict(sparse_test) > 0.5))

In [ ]:
print(confusion_matrix(df_test['is_spoiler'], model.predict(sparse_test) > 0.5))